In [108]:
from collections import defaultdict
from random import random, randint
from glob import glob
from math import log
import argparse
import json
from nltk.corpus import stopwords
from nltk.probability import FreqDist

from nltk.tokenize import TreebankWordTokenizer
kTOKENIZER = TreebankWordTokenizer()
kDOC_NORMALIZER = True

import time
import string
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers import LdaMallet
from gensim.models import LdaModel
import os

import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import os
import string
import nltk
from nltk import word_tokenize
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS
%matplotlib inline
from os import listdir
from os.path import isfile, join
from wordcloud import WordCloud
import gensim
from sklearn.cluster import KMeans
from collections import Counter 
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from wordcloud import WordCloud
import gensim

In [109]:
nlp = spacy.load('en')
stop_words_list = list(spacy.lang.en.STOP_WORDS)
punctuation_list = list(string.punctuation)
punctuation_list.extend('\r')
punctuation_list.extend('\n')
punctuation_list.extend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

In [110]:
def cleaning(element):
    final_list = []
    """ Clean up text. Tokenize, lowercase, and remove punctuation and stopwords """
    #print("Running cleaner")
    # Remove punctuation, symbols (#) and stopwords
    element = str(element).lower()
    new_string = ''
    for char in element:
        if char not in punctuation_list:
            new_string += char
    all_wrds = new_string.split(" ")
    for word in all_wrds:
        if word not in stop_words_list:
            final_list.append(word)
    final_string = ' '.join(final_list)
    return final_string

In [111]:
master= []
def tokenize(str_arg):
    words = str_arg.split()
    return words

def append(row):
    master.append(row)

In [112]:
def topics_gensim(df, city):
    master=[]
    df['cleaned_comments'] = df['comments'].apply(cleaning)
    df['tokens'] = df['cleaned_comments'].apply(tokenize)
    placeholder = df['tokens'].apply(append)
    common_dictionary = Dictionary(master)
    common_corpus = [common_dictionary.doc2bow(text) for text in master]
    lda = LdaModel(common_corpus, num_topics=10, iterations = 5000)
    dictionary={}
    output_name = "gensim-outputs_" + str(city)
    fp = open(output_name, "w", encoding="utf8")
    for i in range(num_topics):
        j = "Topic " + str(i+1)
        words = [common_dictionary[int(i)] for i, p in lda.show_topic(i, topn=50)]
        dictionary[j] = words
    dictionary_json = json.dumps(dictionary)
    fp.write(dictionary_json)
    fp.close()
    

In [113]:
def topics_mallet(df,city):
    master=[]
    df['cleaned_comments'] = df['comments'].apply(cleaning)
    df['tokens'] = df['cleaned_comments'].apply(tokenize)
    placeholder = df['tokens'].apply(append)
    common_dictionary = Dictionary(master)
    common_corpus = [common_dictionary.doc2bow(text) for text in master]
    path = r"/Users/arushij1711/Documents/Sem2/SI630/HW/HW3/mallet-2.0.8/bin/mallet"
    model = LdaMallet(path, corpus=common_corpus, num_topics=10, id2word=common_dictionary, iterations=1000)
    dictionary_1={}
    output_name = "mallet-outputs_" + str(city)
    fp = open("mallet-outputs", "w", encoding="utf8")
    for i in range(10):
        j = "Topic " + str(i+1)
        words = [common_dictionary[int(i)] for i, p in lda.show_topic(i, topn=50)]
        dictionary_1[j] = words
    dictionary_1_json = json.dumps(dictionary_1)
    fp.write(dictionary_1_json)
    fp.close()
    

In [115]:
df_Seattle = pd.read_csv('data/reviews_Seattle.csv')
df_Seattle.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2318,146,2008-09-15,2451,Kevin,1000 times better than staying at a hotel.
1,2318,126302712,2017-01-10,12332845,Jessica,"Our family (two couples, a two year old and an..."
2,2318,140977084,2017-04-01,4789466,Ivan,Top of the list locations we have stayed at! T...
3,2318,147262504,2017-04-25,55817131,Mike,"SUCH an awesome place. Very clean, quiet and s..."
4,2318,161806368,2017-06-18,113604590,Pete,We flew quite a distance to be at our only dau...


In [116]:
df_Seattle['cleaned_comments'] = df_Seattle['comments'].apply(cleaning)
df_Seattle['tokens'] = df_Seattle['cleaned_comments'].apply(tokenize)
df_Seattle.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,cleaned_comments,tokens
0,2318,146,2008-09-15,2451,Kevin,1000 times better than staying at a hotel.,times better staying hotel,"[times, better, staying, hotel]"
1,2318,126302712,2017-01-10,12332845,Jessica,"Our family (two couples, a two year old and an...",family couples year old infant incredible stay...,"[family, couples, year, old, infant, incredibl..."
2,2318,140977084,2017-04-01,4789466,Ivan,Top of the list locations we have stayed at! T...,list locations stayed house perfect families c...,"[list, locations, stayed, house, perfect, fami..."
3,2318,147262504,2017-04-25,55817131,Mike,"SUCH an awesome place. Very clean, quiet and s...",awesome place clean quiet spacious loved,"[awesome, place, clean, quiet, spacious, loved]"
4,2318,161806368,2017-06-18,113604590,Pete,We flew quite a distance to be at our only dau...,flew distance daughters graduation needed plac...,"[flew, distance, daughters, graduation, needed..."


In [129]:
all_words = df_Seattle['cleaned_comments'].str.cat(sep =" ")


In [133]:
myfile = open("bag_of_words_Seattle.txt", "w")
myfile.write(all_words)
myfile.close()

In [118]:
placeholder = df_Seattle['tokens'].apply(append)
#master[:5]

In [119]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(master)
common_corpus = [common_dictionary.doc2bow(text) for text in master]

In [120]:
# Train the model on the corpus.
%time
lda = LdaModel(common_corpus, num_topics=10, iterations = 5000)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [121]:
dictionary={}
fp = open("gensim-outputs", "w", encoding="utf8")
for i in range(10):
    j = "Topic " + str(i+1)
    words = [common_dictionary[int(i)] for i, p in lda.show_topic(i, topn=50)]
    dictionary[j] = words
dictionary_json = json.dumps(dictionary)
fp.write(dictionary_json)
fp.close()

In [122]:
%time
path = r"/Users/arushij1711/Documents/Sem2/SI630/HW/HW3/mallet-2.0.8/bin/mallet"
# os.environ['MALLET_HOME'] = r"\Users\arushij1711\Documents\Sem2\SI630\HW\HW3\mallet-2.0.8"
model = LdaMallet(path, corpus=common_corpus, num_topics=10, id2word=common_dictionary, iterations=1000)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


In [123]:
dictionary_1={}
fp = open("mallet-outputs", "w", encoding="utf8")
for i in range(10):
    j = "Topic " + str(i+1)
    words = [common_dictionary[int(i)] for i, p in lda.show_topic(i, topn=50)]
    dictionary_1[j] = words
dictionary_1_json = json.dumps(dictionary_1)
fp.write(dictionary_1_json)
fp.close()